In [1]:
import os
import torch
import json
import pandas as pd
import re
import openai
from tqdm import tqdm
from openai import AzureOpenAI

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

GPU is not available


In [3]:
import csv
from google.colab import drive
drive.mount('/content/drive')

file_path='/content/drive/MyDrive/Colab Notebooks/comp4211 project/shuffled_A_B_relation_train.csv'
with open(file_path, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    csv_table = list(reader)

df = pd.DataFrame(csv_table)
df['predict']=0
df['EventB'] = df['EventB'].str.replace("[", "", regex=False).str.replace("]", "", regex=False).str.replace("'", "", regex=False).str.strip()
df.head(2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,EventA,EventB,relation,predict
0,PersonX causes trouble,"Is arrested, Thrown in jail",xEffect,0
1,PersonX drops off the radar,"has peace an quiet, gets more sleep",xEffect,0


In [27]:
folder_path = 'dataset'
#df = pd.DataFrame()
pange = pd.DataFrame()

prompt_loc = {
        "prototype": r"/content/drive/MyDrive/Colab Notebooks/comp4211 project/prompt/prototype01.txt",
        "zero-shot": r"/content/drive/MyDrive/Colab Notebooks/comp4211 project/prompt/zero_shot.txt",
        "one-shot": r"/content/drive/MyDrive/Colab Notebooks/comp4211 project/prompt/one_shot.txt",
        "two-shot": r"/content/drive/MyDrive/Colab Notebooks/comp4211 project/prompt/two_shot.txt",
    }

client = AzureOpenAI(
    api_key="0ee412ce839d4a4a99e9abb02ee1eba7",                                              #
    api_version="2023-05-15",                                  #
    azure_endpoint="https://hkust.azure-api.net"
)

def simple_clean(sentence):
    return re.sub(r'[^\w\s]', '', sentence)
def default_clean(sentence):
    return sentence
def categorical_clean(sentence):
    categories = ["oEffect", "oReact", "oWant",  "xAttr","xEffect", "xIntent", "xNeed", "xReact", "xWant"]
    regex = f"{'|'.join(categories)}"
    return re.search(regex, sentence).group() if re.search(regex, sentence) else 'NA'


In [28]:
prompt_list = {name: open(prompt, "r").read() for name, prompt in prompt_loc.items()}

In [29]:
class GPT_QUERY:
    def __init__(self, set_size, table):
        self.set_size = set_size
        self.table = table

    def majority_vote(self, votes):
        votes_table = {}
        for vote in votes:
            if vote in votes_table:
                votes_table[vote] += 1
            else:
                votes_table[vote] = 1
        return max(votes_table, key=votes_table.get)

    def gpt(self, x, prompt, processor):
        response = client.chat.completions.create(
            model="gpt-35-turbo",
            messages=[
                {"role": "user", "content": prompt + x + "\nAnswer:"},
            ]
        )
        #print(prompt + x + "\nAnswer:")
        original = response.choices[0].message.content
        clean_sentence = processor(original)
        return clean_sentence

    def predict(self, prompt_type, processor=categorical_clean, vote=1):    #############
        set_size, table = self.set_size, self.table
        #prompt = prompt_loc[prompt_type]
        prompt = prompt_list[prompt_type]
        with tqdm(total=set_size) as bar:
            for index_A, row_A in table.head(set_size).iterrows():
                sentences = [row_A['EventA'], row_A['EventB']]
                joined_sentence = " == ".join(sentences)
                try:
                    table.at[index_A, 'predict'] = self.majority_vote([ \
                        self.gpt(joined_sentence, prompt, processor) for _ in range(vote)])
                except Exception as error:
                    print(f"An error occurred at index {index_A}:", error)
                bar.update(1)

        table.head(set_size)[['predict', 'relation', 'EventA', 'EventB']].to_csv(f"{prompt_type}.csv")
        print(table.head(set_size)[['predict', 'relation']])

    def estimate(self, ptf=False):
        set_size, table = self.set_size, self.table
        sum, correct = 0, 0
        for index_A, row_A in table.head(set_size).iterrows():
            sum += 1
            if type(row_A['predict']).__name__ == 'str' and type(row_A['relation']).__name__ == 'str' and row_A[
                'predict'].casefold() == row_A['relation'].casefold():
                correct = correct + 1
            else:
                if ptf:
                    print("Wrong prediction:", index_A, row_A['predict'], row_A['relation'])
        return correct / sum


In [24]:
category1 = ["oEffect", "oReact", "oWant",  "xAttr","xEffect", "xIntent", "xNeed", "xReact", "xWant"]
for i in category1:
  first_row = df[df['relation'] == i].iloc[0]
  second_row = df[df['relation'] == i].iloc[1]
  print('EventA:'+first_row['EventA']+'  EventB:'+first_row['EventB']+'  relation:'+first_row['relation'])
  print('EventA:'+second_row['EventA']+'  EventB:'+second_row['EventB']+'  relation:'+second_row['relation'])

EventA:PersonX finishes PersonY's sentence  EventB:PersonY is interrupted, Person Y stops talking  relation:oEffect
EventA:PersonX increases PersonY's efficiency  EventB:puffs out chest with pride, becomes grateful towards personX  relation:oEffect
EventA:PersonX is nowhere to be found  EventB:worried  relation:oReact
EventA:PersonX keeps PersonX's ___ away  EventB:they are being avoided  relation:oReact
EventA:PersonX throws PersonY away  EventB:revenge., their hurt feelings to stop.  relation:oWant
EventA:PersonX makes fun of PersonY  EventB:to give a fitting reply to person X, they respond appropriately and then move away to avoid the situation  relation:oWant
EventA:PersonX is seriously hurt  EventB:hopeless, weak  relation:xAttr
EventA:PersonX buys a new laptop  EventB:in need, capable  relation:xAttr
EventA:PersonX causes trouble  EventB:Is arrested, Thrown in jail  relation:xEffect
EventA:PersonX drops off the radar  EventB:has peace an quiet, gets more sleep  relation:xEffect
E

In [7]:
full_table=df

In [ ]:
#prototype
prompt_list = {name: open(prompt, "r").read() for name, prompt in prompt_loc.items()}
gpt_client = GPT_QUERY(3, full_table)
gpt_client.predict("prototype", vote=3)
gpt_client.estimate()

precision=gpt_client.estimate(True)
print(precision)

In [ ]:
#zero-shot
prompt_list = {name: open(prompt, "r").read() for name, prompt in prompt_loc.items()}
gpt_client = GPT_QUERY(3, full_table)
gpt_client.predict("zero-shot", vote=3)
gpt_client.estimate()

precision=gpt_client.estimate(True)
print(precision)

In [ ]:
#one-shot
prompt_list = {name: open(prompt, "r").read() for name, prompt in prompt_loc.items()}
gpt_client = GPT_QUERY(3, full_table)
gpt_client.predict("one-shot", vote=3)
gpt_client.estimate()

precision=gpt_client.estimate(True)
print(precision)

In [32]:
#two-shot

gpt_client = GPT_QUERY(500, full_table)
gpt_client.predict("two-shot", vote=3)
gpt_client.estimate()

precision=gpt_client.estimate(True)
print(precision)

100%|██████████| 500/500 [26:38<00:00,  3.20s/it]


     predict relation
0    xEffect  xEffect
1    xEffect  xEffect
2      xNeed    xNeed
3      xWant    xWant
4     xReact   xReact
..       ...      ...
495    xAttr    xAttr
496    xAttr    xAttr
497    xAttr    xAttr
498  xIntent    oWant
499  xEffect    xNeed

[500 rows x 2 columns]
Wrong prediction: 10 xIntent xEffect
Wrong prediction: 11 oReact xIntent
Wrong prediction: 17 oEffect xReact
Wrong prediction: 18 oWant xEffect
Wrong prediction: 20 oEffect xEffect
Wrong prediction: 21 xIntent xEffect
Wrong prediction: 22 xIntent xEffect
Wrong prediction: 25 xIntent oReact
Wrong prediction: 26 xIntent xWant
Wrong prediction: 28 xIntent xReact
Wrong prediction: 29 xIntent xReact
Wrong prediction: 30 oReact xIntent
Wrong prediction: 31 xAttr xReact
Wrong prediction: 32 xWant oReact
Wrong prediction: 33 oReact xReact
Wrong prediction: 34 xIntent xWant
Wrong prediction: 35 oReact xReact
Wrong prediction: 36 xWant xEffect
Wrong prediction: 37 xIntent xReact
Wrong prediction: 38 xWant xNeed
W

In [22]:
prompt_list = {name: open(prompt, "r").read() for name, prompt in prompt_loc.items()}
gpt_client = GPT_QUERY(3, full_table)
gpt_client.predict("prototype", vote=3)
gpt_client.estimate()

precision=gpt_client.estimate(True)
print(precision)


# five-shot-CoT-SC
'''
gpt_client = GPT_QUERY(500, full_table)
gpt_client.predict(prompt_list["five-shot-CoT"], vote=3)
gpt_client.estimate()
'''

'''
# five-shot-CoT

gpt_client = GPT_QUERY(250, full_table)
gpt_client.predict(prompt_list["five-shot-CoT"], vote=1)
gpt_client.estimate()

# zero-shot

gpt_client = GPT_QUERY(250, full_table)
gpt_client.predict(prompt_list["zero-shot"])
gpt_client.estimate(ptf=True)

# one-shot

gpt_client = GPT_QUERY(250, full_table)
gpt_client.predict(prompt_list["one-shot"])
gpt_client.estimate(ptf=True)

# five-shot

gpt_client = GPT_QUERY(250, full_table)
gpt_client.predict(prompt_list["five-shot"])
gpt_client.estimate(ptf=True)
'''

100%|██████████| 3/3 [00:09<00:00,  3.04s/it]

   predict relation
0  xIntent  xEffect
1  xEffect  xEffect
2  xIntent    xNeed
Wrong prediction: 0 xIntent xEffect
Wrong prediction: 2 xIntent xNeed
0.3333333333333333


'\n# five-shot-CoT\n\ngpt_client = GPT_QUERY(250, full_table)\ngpt_client.predict(prompt_list["five-shot-CoT"], vote=1)\ngpt_client.estimate()\n\n# zero-shot\n\ngpt_client = GPT_QUERY(250, full_table)\ngpt_client.predict(prompt_list["zero-shot"])\ngpt_client.estimate(ptf=True)\n\n# one-shot\n\ngpt_client = GPT_QUERY(250, full_table)\ngpt_client.predict(prompt_list["one-shot"])\ngpt_client.estimate(ptf=True)\n\n# five-shot\n\ngpt_client = GPT_QUERY(250, full_table)\ngpt_client.predict(prompt_list["five-shot"])\ngpt_client.estimate(ptf=True)\n'